### Determining the outlet of each subbasin
This can either be a *conduit* if the cell with the *highest flow accumulation* value intersects a conduit, or a *sub-basin* if the cell with the *highest flow accumulation* borders another sub-basin.

In [21]:
import rasterio
import rasterio.mask
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping, Point
from shapely.ops import nearest_points

In [23]:


# === Input files ===
subbasins_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\subbasins_drainage_vector_split.gpkg"
conduits_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\SWD Network\updated_conduits_SWMM_new.gpkg"
flow_acc_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\flow_accumulation_subcatchments_clipped.tif"
flow_dir_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\flow_direction_subcatchments_clipped.tif"

# === Load layers ===
gdf = gpd.read_file(subbasins_fp).reset_index().rename(columns={"index": "fid"})
conduits_gdf = gpd.read_file(conduits_fp)

# === Flow direction mapping (GRASS style) ===
grass_dir = {
    1:  (0, 1), 2: (-1, 1), 3: (-1, 0), 4: (-1, -1),
    5:  (0, -1), 6: (1, -1), 7: (1, 0), 8: (1, 1),
}

def world_to_pixel(transform, x, y):
    col, row = ~transform * (x, y)
    return int(row), int(col)

def pixel_to_world(transform, row, col):
    x, y = transform * (col + 0.5, row + 0.5)
    return x, y

# === Open raster data ===
with rasterio.open(flow_acc_fp) as acc_src, rasterio.open(flow_dir_fp) as dir_src:
    acc_data = acc_src.read(1)
    acc_transform = acc_src.transform
    dir_data = dir_src.read(1)
    dir_transform = dir_src.transform

    drains_to_fid = []
    outlet_names = []

    for idx, row in gdf.iterrows():
        shape = mapping(row.geometry.boundary)

        # Mask to find outlet
        try:
            acc_masked, acc_out_transform = rasterio.mask.mask(acc_src, [shape], crop=True, filled=False)
        except ValueError:
            print(f"[fid {row['fid']}] Skipping: No valid raster cells in subbasin.")
            drains_to_fid.append(None)
            outlet_names.append(None)
            continue

        acc_masked = acc_masked[0]
        if np.all(np.isnan(acc_masked)):
            print(f"[fid {row['fid']}] All values are NaN in masked area.")
            drains_to_fid.append(None)
            outlet_names.append(None)
            continue

        # Get outlet point (highest accumulation)
        max_row, max_col = np.unravel_index(np.nanargmax(acc_masked), acc_masked.shape)
        outlet_x, outlet_y = pixel_to_world(acc_out_transform, max_row, max_col)
        outlet_point = Point(outlet_x, outlet_y)

        # Get flow direction at outlet
        row_i, col_i = world_to_pixel(dir_transform, outlet_x, outlet_y)
        if not (0 <= row_i < dir_data.shape[0] and 0 <= col_i < dir_data.shape[1]):
            print(f"[fid {row['fid']}] Outlet point outside flow dir bounds.")
            drains_to_fid.append(None)
            outlet_names.append(None)
            continue

        dir_val = dir_data[row_i, col_i]
        if dir_val not in grass_dir:
            print(f"[fid {row['fid']}] Invalid flow direction value: {dir_val}")
            drains_to_fid.append(None)
            outlet_names.append(None)
            continue

        dr, dc = grass_dir[dir_val]
        new_row, new_col = row_i + dr, col_i + dc
        if not (0 <= new_row < dir_data.shape[0] and 0 <= new_col < dir_data.shape[1]):
            print(f"[fid {row['fid']}] Downstream point out of bounds.")
            drains_to_fid.append(None)
            outlet_names.append(None)
            continue

        # Convert downstream point to world coords
        new_x, new_y = pixel_to_world(dir_transform, new_row, new_col)
        downstream_point = Point(new_x, new_y)

        # === First try to find intersecting conduit ===
        conduit_match = None
        for _, conduit in conduits_gdf.iterrows():
            if conduit.geometry.intersects(downstream_point.buffer(10)):
                conduit_match = conduit['Name']
                break

        if conduit_match:
            outlet_names.append(conduit_match)
            drains_to_fid.append(None)
            print(f"[fid {row['fid']}] Drains to conduit: {conduit_match}")
            continue

        # === Else try to find downstream subcatchment ===
        found_fid = None
        found_name = None
        for _, other_row in gdf.iterrows():
            if other_row['fid'] != row['fid'] and other_row.geometry.contains(downstream_point):
                found_fid = other_row['fid']
                found_name = other_row['Name']
                break

        drains_to_fid.append(found_fid)
        outlet_names.append(found_name)
        print(f"[fid {row['fid']}] Drains to subcatchment: {found_name if found_name else 'None'}")

# Add results to GeoDataFrame
gdf["drains_to"] = drains_to_fid
gdf["Outlet"] = outlet_names

# Save output
gdf.to_file(r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_with_outlet_logic_buffer10_SWMM.gpkg", driver="GPKG")
print("✅ Done! Saved with both 'drains_to' and 'Outlet' fields.")


[fid 0] Drains to subcatchment: S_4
[fid 1] Drains to conduit: C_0
[fid 2] Drains to subcatchment: S_42
[fid 3] Drains to subcatchment: S_3
[fid 4] Drains to conduit: C_0
[fid 5] Drains to subcatchment: S_1
[fid 6] Drains to conduit: C_0
[fid 7] Drains to conduit: C_0
[fid 8] Drains to subcatchment: S_42
[fid 9] Drains to subcatchment: S_42
[fid 10] Drains to conduit: C_0
[fid 11] Drains to subcatchment: S_45
[fid 12] Drains to conduit: C_0
[fid 13] Drains to subcatchment: S_45
[fid 14] Drains to conduit: C_1
[fid 15] Drains to conduit: C_1
[fid 16] Drains to conduit: C_1
[fid 17] Drains to subcatchment: S_27
[fid 18] Drains to subcatchment: S_27
[fid 19] Drains to conduit: C_1
[fid 20] Drains to conduit: C_47
[fid 21] Drains to subcatchment: S_33
[fid 22] Drains to conduit: C_47
[fid 23] Drains to subcatchment: S_53
[fid 24] Drains to subcatchment: S_53
[fid 25] Drains to subcatchment: S_33
[fid 26] Drains to subcatchment: S_64
[fid 27] Drains to subcatchment: S_27
[fid 28] Drains to 

### Giving LULC layer binary values
1 = paved, 0 = unpaved

In [24]:
import geopandas as gpd

# === File paths ===
input_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\Esa_worldcover_Nakuru_town_clipped.gpkg"
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\\esa_worldcover_binary.gpkg"

# === Read the GPKG ===
gdf = gpd.read_file(input_fp)

# === Create binary column: 50 → 1, else → 0 ===
# Replace 'landcover' with the actual column name in your file
gdf['landclass_bin'] = gdf['DN'].apply(lambda x: 1 if x == 50 else 0)

# === Save to new GPKG ===
gdf.to_file(output_fp, driver="GPKG")

print("✅ Done! Output saved with binary classification.")



✅ Done! Output saved with binary classification.


### link paved % to subcatchments

In [25]:
import geopandas as gpd

# === File paths ===
subcatchments_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_with_outlet_logic_buffer10_SWMM.gpkg"
paved_unpaved_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\esa_worldcover_binary.gpkg"
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_with_pct_paved_SWMM.gpkg"

# === Read layers ===
subcatchments = gpd.read_file(subcatchments_fp)
paved_unpaved = gpd.read_file(paved_unpaved_fp)

# === Ensure same CRS ===
paved_unpaved = paved_unpaved.to_crs(subcatchments.crs)

# === Clip paved/unpaved to each subcatchment ===
pct_paved_list = []

for _, sc in subcatchments.iterrows():
    sc_geom = sc.geometry
    sc_area = sc_geom.area  # Total area of subcatchment
    
    # Clip paved/unpaved to current subcatchment geometry
    clipped = paved_unpaved.clip(sc_geom)
    
    # Sum the paved areas (where value = 1)
    paved_area = clipped[clipped['landclass_bin'] == 1].geometry.area.sum()

    # Calculate the percentage of paved area
    pct_paved = int(round((paved_area / sc_area) * 100)) if sc_area > 0 else 0
    pct_paved_list.append(pct_paved)

# === Add percentage paved to subcatchments ===
subcatchments["Imperv"] = pct_paved_list

# === Save the updated subcatchments ===
subcatchments.to_file(output_fp, driver="GPKG")
print("✅ Done! Subcatchments saved with 'Imperv' column.")


✅ Done! Subcatchments saved with 'Imperv' column.


### Determining the slope for each subcatchment

In [26]:
!pip install rasterstats

In [27]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rasterstats import zonal_stats

# === File path ===
dem_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\2. QGIS + Wflow\SWMM\subcatchments generation\DEM_depressionless_SWMM.tif"
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\Subbasins_drainage_imperv_slope.gpkg"
subcatchments_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_with_pct_paved_SWMM.gpkg"

# === Step 1: Calculate slope from DEM ===
with rasterio.open(dem_fp) as dem_src:
    dem = dem_src.read(1)  # Read DEM as array
    transform = dem_src.transform
    crs = dem_src.crs
    res_x, res_y = dem_src.res

# Calculate slope
x_grad, y_grad = np.gradient(dem, res_x, res_y)
slope_rad = np.arctan(np.sqrt(x_grad**2 + y_grad**2))
slope_deg = np.degrees(slope_rad)

# Save slope raster temporarily
slope_tmp_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\slope_tmp.tif"
with rasterio.open(
    slope_tmp_fp, "w",
    driver="GTiff",
    height=slope_deg.shape[0],
    width=slope_deg.shape[1],
    count=1,
    dtype=slope_deg.dtype,
    crs=crs,
    transform=transform
) as dst:
    dst.write(slope_deg, 1)

# === Step 2: Calculate average slope per subcatchment ===
subcatchments = gpd.read_file(subcatchments_fp)

# Calculate mean slope using zonal_stats
zs = zonal_stats(
    subcatchments_fp, slope_tmp_fp,
    stats="mean",
    geojson_out=True
)

# Extract mean slope and attach to GeoDataFrame
mean_slope_values = [feature['properties']['mean'] for feature in zs]
subcatchments['Slope'] = [round(val, 2) if val is not None else None for val in mean_slope_values]

# === Step 3: Save the subcatchments with the avg_slope_deg column ===
subcatchments.to_file(output_fp, driver="GPKG")
print(f"✅ Done! Subcatchments with average slope saved to {output_fp}")



✅ Done! Subcatchments with average slope saved to C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\Subbasins_drainage_imperv_slope.gpkg


In [28]:
import geopandas as gpd

# Load your subcatchments layer
subcatchments_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\Subbasins_drainage_imperv_slope.gpkg"
output_fp = r"C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_SWMM_ready.gpkg"

gdf = gpd.read_file(subcatchments_fp)

# Remove 'drains_to' and 'DN' columns if present
for col in ['drains_to', 'DN']:
    if col in gdf.columns:
        gdf = gdf.drop(columns=col)

# Add/overwrite columns as specified
gdf['RainGage'] = 1
gdf['Area'] = gdf.geometry.area / 10_000  # area in hectares

# If area is 0, set to 1
gdf.loc[gdf['Area'] == 0, 'Area'] = 1

gdf['Width'] = 100
gdf['CurbLen'] = 0
gdf['SnowPack'] = None
gdf['N_Imperv'] = 0.01
gdf['N_Perv'] = 0.05
gdf['S_Imperv'] = 0.5
gdf['S_Perv'] = 0.5
gdf['PctZero'] = 25
gdf['RouteTo'] = "OUTLET"
gdf['PctRouted'] = 100
gdf['InfMethod'] = "HORTON"
gdf['SuctHead'] = None
gdf['Conductiv'] = None
gdf['InitDef'] = None
gdf['MaxRate'] = 5
gdf['MinRate'] = 0.5
gdf['Decay'] = 0.001
gdf['DryTime'] = 48
gdf['MaxInf'] = 10
gdf['CurveNum'] = None
gdf['Annotation'] = None

# Ensure 'Slope' column is included (should already be present)
if 'Slope' not in gdf.columns:
    raise ValueError("Slope column is missing from the input file.")

# If slope is 0, set to 0.01
gdf.loc[gdf['Slope'] == 0, 'Slope'] = 0.01

# Save to new file
gdf.to_file(output_fp, driver="GPKG")
print(f"✅ Done! Saved with all SWMM columns to {output_fp}")

✅ Done! Saved with all SWMM columns to C:\Users\jmsch\OneDrive\Documenten\Studie\Civiele Techniek\Environmental Engineering\Year 2\Afstuderen\NBS Nakuru Kenia\5. Python\Python_QGIS\subcatchments_SWMM_ready.gpkg
